In [1]:
import matplotlib.pyplot as plt
import pandas as pd


In [2]:
DATA_PATH = "/Users/howardtangkulung/code/personal_projects/financial-concepts/data/"
df = pd.read_csv(DATA_PATH + 'treasury_yield_curve.csv')
df

,Date,3 Mo,6 Mo,1 Yr,2 Yr,3 Yr,5 Yr,7 Yr,10 Yr,20 Yr,30 Yr
0,1991-01-02,6.66,6.73,6.74,7.08,7.30,7.59,7.90,7.97,NaN,8.14
1,1991-01-16,6.22,6.46,6.61,7.21,7.47,7.83,8.09,8.24,NaN,8.40
2,1991-01-31,6.37,6.49,6.51,7.05,7.30,7.62,7.89,8.03,NaN,8.21
3,1991-02-14,6.02,6.10,6.19,6.77,6.98,7.38,7.67,7.80,NaN,7.99
4,1991-03-01,6.27,6.38,6.52,7.18,7.41,7.78,8.00,8.12,NaN,8.28
...,...,...,...,...,...,...,...,...,...,...,...
830,2023-10-19,5.60,5.56,5.44,5.14,5.01,4.95,5.00,4.98,5.30,5.11
831,2023-11-02,5.54,5.50,5.38,4.98,4.78,4.65,4.68,4.67,4.99,4.82
832,2023-11-16,5.51,5.38,5.23,4.83,4.59,4.43,4.47,4.45,4.82,4.63
833,2023-12-01,5.43,5.33,5.05,4.56,4.31,4.14,4.22,4.22,4.58,4.40


In [3]:
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)
display(df)

,3 Mo,6 Mo,1 Yr,2 Yr,3 Yr,5 Yr,7 Yr,10 Yr,20 Yr,30 Yr
Date,,,,,,,,,,
1991-01-02,6.66,6.73,6.74,7.08,7.30,7.59,7.90,7.97,NaN,8.14
1991-01-16,6.22,6.46,6.61,7.21,7.47,7.83,8.09,8.24,NaN,8.40
1991-01-31,6.37,6.49,6.51,7.05,7.30,7.62,7.89,8.03,NaN,8.21
1991-02-14,6.02,6.10,6.19,6.77,6.98,7.38,7.67,7.80,NaN,7.99
1991-03-01,6.27,6.38,6.52,7.18,7.41,7.78,8.00,8.12,NaN,8.28
...,...,...,...,...,...,...,...,...,...,...
2023-10-19,5.60,5.56,5.44,5.14,5.01,4.95,5.00,4.98,5.30,5.11
2023-11-02,5.54,5.50,5.38,4.98,4.78,4.65,4.68,4.67,4.99,4.82
2023-11-16,5.51,5.38,5.23,4.83,4.59,4.43,4.47,4.45,4.82,4.63


In [4]:
maturity = {
    "3 Mo": pd.DateOffset(months=3),
    "6 Mo": pd.DateOffset(months=6),
    "1 Yr": pd.DateOffset(years=1),
    "2 Yr": pd.DateOffset(years=2),
    "3 Yr": pd.DateOffset(years=3),
    "5 Yr": pd.DateOffset(years=5),
    "7 Yr": pd.DateOffset(years=7),
    "10 Yr": pd.DateOffset(years=10),
    "20 Yr": pd.DateOffset(years=20),
    "30 Yr": pd.DateOffset(years=30)
}

# Set as 3d array

current_dates = []
yields = []
maturity_dates = []

for date in df.index:
    for key, value in maturity.items():
        current_dates.append(date)
        yields.append(df[key][date])
        maturity_dates.append(date + value)

In [5]:
import numpy as np
import pandas as pd
from scipy.interpolate import griddata
import plotly.graph_objects as go

# Convert dates to ordinal numbers
current_dates_num = pd.to_datetime(current_dates).map(pd.Timestamp.toordinal)
maturity_dates_num = pd.to_datetime(maturity_dates).map(pd.Timestamp.toordinal)

# Create grid for interpolation
grid_x, grid_y = np.mgrid[
    current_dates_num.min():current_dates_num.max():100j,
    maturity_dates_num.min():maturity_dates_num.max():100j
]

# Perform interpolation
grid_z = griddata(
    (current_dates_num, maturity_dates_num),
    yields,
    (grid_x, grid_y),
    method='linear'
)

# Handle NaN values
grid_z = np.nan_to_num(grid_z)

# Create the surface plot
fig = go.Figure(data=[go.Surface(x=grid_x, y=grid_y, z=grid_z)])

# Define tick positions and labels
tickvals_x = np.linspace(current_dates_num.min(), current_dates_num.max(), 10)
tickvals_y = np.linspace(maturity_dates_num.min(), maturity_dates_num.max(), 10)
ticktext_x = [pd.Timestamp.fromordinal(int(val)).strftime('%Y-%m') for val in tickvals_x]
ticktext_y = [pd.Timestamp.fromordinal(int(val)).strftime('%Y-%m') for val in tickvals_y]

# Update layout
fig.update_layout(
    title='Yield Curve Surface Plot',
    scene=dict(
        xaxis=dict(
            title='Current Date',
            tickmode='array',
            tickvals=tickvals_x,
            ticktext=ticktext_x
        ),
        yaxis=dict(
            title='Maturity Date',
            tickmode='array',
            tickvals=tickvals_y,
            ticktext=ticktext_y
        ),
        zaxis_title='Yield'
    ),
    width=800,
    height=800
)


fig.show()


In [6]:
# plot the 3M yield curve

fig = go.Figure()
fig.add_trace(go.Scatter(x=df.index, y=df['3 Mo'], mode='lines', name='3 Mo'))
fig.add_trace(go.Scatter(x=df.index, y=df['1 Yr'], mode='lines', name='1 Yr'))
fig.add_trace(go.Scatter(x=df.index, y=df['5 Yr'], mode='lines', name='5 Yr'))
fig.add_trace(go.Scatter(x=df.index, y=df['10 Yr'], mode='lines', name='10 Yr'))

fig.update_layout(
    title='Yield Curve',
    xaxis_title='Date',
    yaxis_title='Yield',
    width=800, height=600,
    legend_title='Maturity',
    colorway=["#FFFFED", "#FFD000", "#FFA500", "#FF2600"]
    )


In [30]:
# plot a normal yield curve
NORMAL_YIELD_DATE = "2022-03-02"
fig = go.Figure()

# Create the normal yield curve
normal_yields = df.loc[NORMAL_YIELD_DATE]

x = [pd.Timestamp(NORMAL_YIELD_DATE) + maturity[key] for key in normal_yields.keys()]
fig.add_trace(go.Scatter(x=x, y=normal_yields, name='Yield Curve'))
fig.update_layout(
    xaxis=dict(
        tickmode='array',
        tickvals=x,
        ticktext=[key if key != "6 Mo" else " " for key in normal_yields.keys()],
        tickangle=-90
    ),
    title=f'Normal Yield Curve on {NORMAL_YIELD_DATE}',
    xaxis_title='Maturity',
    yaxis_title='Yield',
    width=600, height=400
)

In [8]:
# plot a normal yield curve
INVERTED_YIELD_DATE = "2007-03-15"
fig = go.Figure()

# Create the normal yield curve
normal_yields = df.loc[INVERTED_YIELD_DATE]
fig.add_trace(go.Scatter(x=normal_yields.index, y=normal_yields, name='Yield Curve'))
fig.add_trace(go.Scatter(x=normal_yields.index, y=normal_yields, mode='lines', name='Yield Curve'))
fig.update_layout(
    title='Inverted Yield Curve',
    xaxis_title='Maturity',
    yaxis_title='Yield',
    width=600, height=400
)

In [9]:
import yfinance as yahooFinance

spy_df = yahooFinance.Ticker("SPY").history(start='1990-01-01', end='2023-03-01')[::5]["Close"]
spy_df

Date
1993-01-29 00:00:00-05:00     24.684105
1993-02-05 00:00:00-05:00     25.263451
1993-02-12 00:00:00-05:00     25.052790
1993-02-22 00:00:00-05:00     24.561226
1993-03-01 00:00:00-05:00     24.877220
                                ...    
2023-01-24 00:00:00-05:00    391.723816
2023-01-31 00:00:00-05:00    397.870850
2023-02-07 00:00:00-05:00    406.396301
2023-02-14 00:00:00-05:00    403.900330
2023-02-22 00:00:00-05:00    390.098907
Name: Close, Length: 1515, dtype: float64

In [10]:
# (1 + x / 100) = now_day / previous_day
ratio = (spy_df / spy_df.shift(1)).dropna() - 1

# get 30 days window for moving average
moving_average = ratio.rolling(window=30).mean()
moving_average

Date
1993-02-05 00:00:00-05:00         NaN
1993-02-12 00:00:00-05:00         NaN
1993-02-22 00:00:00-05:00         NaN
1993-03-01 00:00:00-05:00         NaN
1993-03-08 00:00:00-05:00         NaN
                               ...   
2023-01-24 00:00:00-05:00    0.003708
2023-01-31 00:00:00-05:00    0.002160
2023-02-07 00:00:00-05:00    0.003443
2023-02-14 00:00:00-05:00    0.003337
2023-02-22 00:00:00-05:00    0.001197
Name: Close, Length: 1514, dtype: float64

In [11]:
from plotly.subplots import make_subplots

SHORT_TERM_MATURITY = "3 Mo"
LONG_TERM_MATURITY = "10 Yr"

df["spread"] = df[LONG_TERM_MATURITY] - df[SHORT_TERM_MATURITY]
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(x=df.index, y=df["spread"], mode='lines', name=f'{LONG_TERM_MATURITY} - {SHORT_TERM_MATURITY} spread'), secondary_y=False)
fig.add_trace(go.Scatter(x=spy_df.index, y=moving_average, mode='lines', name='SPY'), secondary_y=True)
fig.update_layout(
    title='Yield Spread',
    xaxis_title='Date',
    yaxis_title='Spread',
    width=800, height=500,
    legend_title='Spread'
)
